# Websrapping a Paginas de Supermercados en Linea Mexicanos

Utilizaremos varias herramientas
- bs4 (Beautiful Soup)
- request 
- selenium 
- msedge.selenium_tools


El flujo es el siguiente: 
1. Ingresamos a la página web con el url que vamos a construir -> en este caso esta predeterminado pero ustedes pueden crear una funcion para automatizarlo quiza leyendo los valores de una BD o un JSON .... 
2.  Ahora relizaremos el scrapping -> identificaremos 3 valores importantes que vamos a necesitar <b>Nombre</b> , <b>Precio</b> , <b>Url</b> donde dependera como fue el diseño de las mismas para poder extrar la informacion
3. Procesaremos la información para posteriormente agregarla a un diccionario , y finalmente serializarla con un JSON a alguna base de datos, en nuestro caso utilizaremos <b>MONGO DB</b>
4. El scrapping esta listo, lo que falta ahora es ¿Qué hacer con esta informacion?

### Webscrapping Aurrera

Aqui tenemos ademas una cheatsheet de como realizar las principales utilidades de <b>Selenium</b>

 ['url cheatsheets'](https://www.automatetheplanet.com/selenium-webdriver-java-cheat-sheet/)

In [1]:
# Url de Bodega Aurrera
link = "https://www.bodegaaurrera.com.mx/productos?Ntt=comida%20gato" 

In [2]:
# obtener el html de la pagina web link
import requests
from bs4 import BeautifulSoup

page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

# imprimir el html de la pagina web link
print(soup.prettify())


<html>
 <head>
  <title>
   Access Denied
  </title>
 </head>
 <body>
  <h1>
   Access Denied
  </h1>
  You don't have permission to access "http://www.bodegaaurrera.com.mx/productos?" on this server.
  <p>
   Reference #18.ef72217.1679273174.b678360
  </p>
 </body>
</html>



MMM nos marca un error de permisos --> 

Intentaremos realizar la busqueda con un request, pero como es una página con una restriccion de navegador y ademas <b>dinamica</b> , entonces buscaremos con SELENIUM

Para esta sección y las demás que veamos esta importacion de librerias, las dejare marcadas por si solo necesitan ejecutar ese bloque de codigo

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from msedge.selenium_tools import EdgeOptions
from msedge.selenium_tools import Edge
import time
PATH = "./msedgedriver.exe"

options = EdgeOptions()
options.use_chromium = True
options.headless = True
options.add_experimental_option(
    "excludeSwitches", ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
driver = Edge(options=options)


C:\Users\Adolfo\AppData\Local\Temp\ipykernel_7012\2784977948.py:18: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


In [4]:
driver = webdriver.Edge(PATH)
driver.get(link)
html = ""
# esperar 6 segundos -> esto es para que cargue la pagina web
time.sleep(6)
# obtener el html de la pagina web link
html = driver.page_source
driver.quit()

#### Nota: Debido a que tienen mejores sistemas de seguridad anti scrapers , no pudimos realizar nuestro trabajo con Aurrera, bien ahi grupo Walmart

In [5]:
# contar el numero de divs que hay en el siguiente xpath //*[@id="scrollContainer"]/section/div[1]/div[3]/div[2]/div[1]/div
# divs = driver.find_element(by=By.XPATH, value='//*[@id="scrollContainer"]/section/div[1]/div[3]/div[2]/div[1]/div')

# datos = divs.find_elements(By.TAG_NAME, "div")

# limpieza = []
# productos = []
# for x in datos:
#    limpieza.append(x.get_attribute("innerHTML"))


In [6]:
# crear diccionario con los datos de los productos
# import re
# nombre_temporal = ""
# for x in limpieza:
#    soup = BeautifulSoup(x, 'html.parser')


#    if(soup.find('a') != None):

#        precio = (soup.find(attrs={"data-automation-id" : "sale-price"}))
#        if(precio != None):
#            precio = precio.text
#            nombre = (soup.find(attrs={"data-automation-id" : "product-name"}).text)

#            producto = {
#                "nombre": nombre,
#                "precio": precio,
#                "url": "https://www.bodegaaurrera.com.mx"+soup.find('a').get('href')
#            }

#            if nombre != nombre_temporal:
#                productos.append(producto)
#                nombre_temporal = nombre
# driver.quit()


## WebScrapping para Chedraui

In [7]:
# con el webdriver de edge, abrir la pagina web link
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from msedge.selenium_tools import EdgeOptions
from msedge.selenium_tools import Edge
import time
import datetime
PATH = "./msedgedriver.exe"


# Crearemos un diccionario con los productos que queremos buscar

comida_gatoC = {
    "nombre": "Comida para gato",
    "url": "https://www.chedraui.com.mx/gatos?_q=gatos&map=ft"
}

mayonesaC = {
    "nombre": "Mayonesa",
    "url": "https://www.chedraui.com.mx/mayonesa?_q=mayonesa&map=ft"
}


diccionarioC = {
    "comida_gato": comida_gatoC,
    "mayonesa": mayonesaC
}

driver = webdriver.Edge(PATH)
#importante definir el tamaño de la ventana a maximizar por el tema de el CSS responsive
driver.maximize_window()
pChedraui = []


In [8]:
# obtener los precios de los productos de la pagina web link
for key, value in diccionarioC.items():
    driver.get(value["url"])
    html = ""
    # esperar 6 segundos
    time.sleep(6)
    # obtener el html de la pagina web link
    html = driver.page_source

    productosChedraui = driver.find_element(By.ID, "gallery-layout-container")
    productosChedraui = productosChedraui.find_elements(By.TAG_NAME, "div")
    contador = 1

    nombre_complete = ""
    precio_complete = ""

    for x in productosChedraui:

        elementoPrecio = "/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div/div[3]/div/div/div/div[" + str(
            contador)+"]/section/a/article/div[6]/div/div/span/span"
        elementoNombre = "/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div/div[3]/div/div/div/div["+str(
            contador)+"]/section/a/article/div[5]/h3/span"
        elementoUrl = "/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div/div[3]/div/div/div/div["+str(
            contador)+"]/section/a"

        try:
            precio = x.find_element(By.XPATH, elementoPrecio).text
        except Exception as e:
            precio = ""

        try:
            nombre = x.find_element(By.XPATH, elementoNombre).text
        except:
            nombre = ""
        try:
            url = x.find_element(By.XPATH, elementoUrl).get_attribute("href")
        except:
            url = ""

        if (nombre != "" and nombre != None):
            # print("Nombre: "+ nombre)
            nombre_complete = nombre
        if (precio != "" and precio != None):
            # print("Precio: "+ precio)
            precio_complete = precio
        if (url != "" and url != None):
            # print("Url: "+ url)
            url_complete = url

        if (nombre_complete == "" or precio_complete == "" or url_complete == ""):
            continue

        producto = {
            "nombre": nombre_complete,
            "busqueda": value["nombre"],
            "precio": precio_complete,
            "url": url_complete,
            "tienda": "Chedraui",
            "fecha": datetime.datetime.now().strftime("%Y-%m-%d")

        }

        contador = contador + 1

        pChedraui.append(producto)
    print(pChedraui)
contador = 1
driver.quit()


[{'nombre': 'Purina Cat Chow Defenseplus Gatitos Sabor Pescado Carne y Leche 1.5kg', 'busqueda': 'Comida para gato', 'precio': '$128.00', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-defenseplus-gatitos-sabor-pescado-carne-y-leche-15kg-3074427/p', 'tienda': 'Chedraui', 'fecha': '2023-03-19'}, {'nombre': 'Purina Cat Chow Defense Plus para Gatos Adultos Sabor Carne 3kg', 'busqueda': 'Comida para gato', 'precio': '$277.00', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-defense-plus-para-gatos-adultos-sabor-carne-3kg-3709060/p', 'tienda': 'Chedraui', 'fecha': '2023-03-19'}, {'nombre': 'Purina Cat Chow Delimix con Defenseplus Gatos Adultos 1.5kg', 'busqueda': 'Comida para gato', 'precio': '$119.00', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-delimix-con-defenseplus-gatos-adultos-15kg-3258972/p', 'tienda': 'Chedraui', 'fecha': '2023-03-19'}, {'nombre': 'Purina Cat Chow Defensehydro Adultos Pescado Alimento Húmedo Sobre 85g', 'busqueda': 'Comida para gato', 'precio': '$1

In [9]:
pChedraui

[{'nombre': 'Purina Cat Chow Defenseplus Gatitos Sabor Pescado Carne y Leche 1.5kg',
  'busqueda': 'Comida para gato',
  'precio': '$128.00',
  'url': 'https://www.chedraui.com.mx/purina-cat-chow-defenseplus-gatitos-sabor-pescado-carne-y-leche-15kg-3074427/p',
  'tienda': 'Chedraui',
  'fecha': '2023-03-19'},
 {'nombre': 'Purina Cat Chow Defense Plus para Gatos Adultos Sabor Carne 3kg',
  'busqueda': 'Comida para gato',
  'precio': '$277.00',
  'url': 'https://www.chedraui.com.mx/purina-cat-chow-defense-plus-para-gatos-adultos-sabor-carne-3kg-3709060/p',
  'tienda': 'Chedraui',
  'fecha': '2023-03-19'},
 {'nombre': 'Purina Cat Chow Delimix con Defenseplus Gatos Adultos 1.5kg',
  'busqueda': 'Comida para gato',
  'precio': '$119.00',
  'url': 'https://www.chedraui.com.mx/purina-cat-chow-delimix-con-defenseplus-gatos-adultos-15kg-3258972/p',
  'tienda': 'Chedraui',
  'fecha': '2023-03-19'},
 {'nombre': 'Purina Cat Chow Defensehydro Adultos Pescado Alimento Húmedo Sobre 85g',
  'busqueda'

In [10]:
# guardar los datos de los productos en un archivo json
import json
with open('productosChedraui.json', 'w') as outfile:
    json.dump(pChedraui, outfile)


In [11]:
# TODO: REALIZAR LA Busqueda tambien con Justo
# urlJusto = "https://justo.mx/search?q=Gatos"


### Webscraping Soriana

In [12]:
options = EdgeOptions()
options.use_chromium = True
options.headless = True
options.add_experimental_option(
    "excludeSwitches", ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
driver = Edge(options=options)


driver = webdriver.Edge(PATH)
driver.maximize_window()

pSoriana = []

driver.get(
    "https://www.soriana.com/buscar?q=gatos&cid=&search-button=&page=2&cref=0&view=grid")
html = ""
# esperar 5 segundos
time.sleep(6)
# obtener el html de la pagina web link
html = driver.page_source
# iterar 20 veces para obtener los 20 productos de la pagina web
for x in range(20):
    datos = "/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[" + \
        str(x)+"]/div[1]/div/div[1]/a/img"

    url = "/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[" + \
        str(x)+"]/div[1]/div/div[1]/a"

    try:

        objeto = driver.find_element(By.XPATH, datos)

        url = driver.find_element(By.XPATH, url)

        texto_json = (objeto.get_attribute("data-cbt"))
        ulr = url.get_attribute("href")

        objeto_json = json.loads(texto_json)
        print(objeto_json)
        # Acceder a la lista de productos
        productos = objeto_json['ecommerce']['click']['products']

        # Imprimir los productos
        # print(productos)

        a = productos[0]
        nombre_producto = productos[0]['name']
        precio_producto = productos[0]['price']
        url_producto = ulr

        # print("Precio", a['price'])
        # print("Nombre", a['name'])
        # print("Url", ulr)

        producto = {
            "precio": a['price'],
            "busqueda": "Comida para gato",
            "nombre": a['name'],
            "url": ulr,
            "tienda": "Soriana",
            "fecha": datetime.datetime.now().strftime("%Y-%m-%d")
        }

        pSoriana.append(producto)

    except Exception as e:
        objeto = ""
        print(e)
time.sleep(2)
driver.quit()


C:\Users\Adolfo\AppData\Local\Temp\ipykernel_7012\2969844606.py:8: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[0]/div[1]/div/div[1]/a/img"}
  (Session info: MicrosoftEdge=111.0.1661.44)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[1]/div[1]/div/div[1]/a/img"}
  (Session info: MicrosoftEdge=111.0.1661.44)

{'event': 'showProductClick', 'eventTypes': 'click', 'ecommerce': {'click': {'actionField': {'list': 'Search Results'}, 'products': [{'category': 'Mascotas', 'name': 'Alimento para gato Whiskas esterlizado 1.5kg', 'price': 109, 'id': '11586908', 'variant': '', 'brand': 'Whiskas', 'position': 0}]}, 'currencyCode': 'MXN'}}
{'event': 'showProductClick', 'eventTypes': 'click', 'ecommerce': {'click': {'actionField': {'list': 'Search Results'}, 'products': [{'category': 'Mascotas', 'name': 'Alimento Para Gato Whiskas 3 Kg', 'price': 239, 'id': '115

En este caso especifico no haremos uso de un diccionario para iterar la información , debido justamente a que presentamos un error en nuestra ejecucion de un error con un forbidden :_c 

In [13]:
options = EdgeOptions()
options.use_chromium = True
options.headless = True
options.add_experimental_option(
    "excludeSwitches", ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
driver = Edge(options=options)


driver = webdriver.Edge(PATH)
driver.maximize_window()


driver.get(
    "https://www.soriana.com/buscar?q=mayonesa&cid=&search-button=&page=2&cref=0&view=grid")
html = ""
# esperar 5 segundos
time.sleep(6)
# obtener el html de la pagina web link
html = driver.page_source
# iterar 20 veces para obtener los 20 productos de la pagina web
for x in range(20):
    datos = "/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[" + \
        str(x)+"]/div[1]/div/div[1]/a/img"

    url = "/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[" + \
        str(x)+"]/div[1]/div/div[1]/a"

    try:

        objeto = driver.find_element(By.XPATH, datos)
        print(objeto)
        url = driver.find_element(By.XPATH, url)

        texto_json = (objeto.get_attribute("data-cbt"))
        ulr = url.get_attribute("href")

        objeto_json = json.loads(texto_json)

        # Acceder a la lista de productos
        productos = objeto_json['ecommerce']['click']['products']



        a = productos[0]
        nombre_producto = productos[0]['name']
        precio_producto = productos[0]['price']
        url_producto = ulr

        producto = {
            "precio": a['price'],
            "busqueda": "Mayonesa",
            "nombre": a['name'],
            "url": ulr,
            "tienda": "Soriana",
            "fecha": datetime.datetime.now().strftime("%Y-%m-%d")
        }

        pSoriana.append(producto)

    except Exception as e:
        objeto = ""
        print(e)
time.sleep(2)
driver.quit()


C:\Users\Adolfo\AppData\Local\Temp\ipykernel_7012\578507552.py:8: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[0]/div[1]/div/div[1]/a/img"}
  (Session info: MicrosoftEdge=111.0.1661.44)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[1]/div[1]/div/div[1]/a/img"}
  (Session info: MicrosoftEdge=111.0.1661.44)

<selenium.webdriver.remote.webelement.WebElement (session="d23ddb6ae8af07b617de88174890482d", element="8d36ed83-47f9-422b-9f59-fc80bee76881")>
<selenium.webdriver.remote.webelement.WebElement (session="d23ddb6ae8af07b617de88174890482d", element="b1a58c6b-6bb2-4a9e-ab82-549c3885ae63")>
<selenium.webdriver.remote.webelement.WebElement (session="d23ddb6ae8af07b617de88174890482d", element="71dc4a44-715e-48c4-b812-9b9f7d039659")>
<selenium.webdriver.remote.webelement.WebElement (session="d23ddb6ae8af07b617de88174890482d", element="7f1516e5-32cc-40

In [14]:
pSoriana

[{'precio': 109,
  'busqueda': 'Comida para gato',
  'nombre': 'Alimento para gato Whiskas esterlizado 1.5kg',
  'url': 'https://www.soriana.com/alimento-para-gato-whiskas-esterlizado-1.5kg/11586908.html',
  'tienda': 'Soriana',
  'fecha': '2023-03-19'},
 {'precio': 239,
  'busqueda': 'Comida para gato',
  'nombre': 'Alimento Para Gato Whiskas 3 Kg',
  'url': 'https://www.soriana.com/alimento-para-gato-whiskas-3-kg/11586931.html',
  'tienda': 'Soriana',
  'fecha': '2023-03-19'},
 {'precio': 113,
  'busqueda': 'Comida para gato',
  'nombre': 'Alimento Para Gato Felix Sabor Atun',
  'url': 'https://www.soriana.com/alimento-para-gato-felix-sabor-atun/11590937.html',
  'tienda': 'Soriana',
  'fecha': '2023-03-19'},
 {'precio': 95.5,
  'busqueda': 'Comida para gato',
  'nombre': 'Nucat Alimento Para Gato 1.8 Kilos',
  'url': 'https://www.soriana.com/nucat-alimento-para-gato-1.8-kilos/11254658.html',
  'tienda': 'Soriana',
  'fecha': '2023-03-19'},
 {'precio': 118,
  'busqueda': 'Comida para

In [15]:
# guardar los datos de los productos en un archivo json

import json
with open('productosSoriana.json', 'w') as outfile:
    json.dump(pSoriana, outfile)


----------------------


Podemos agregar una capa de limpieza de datos opcionalmente.... 

In [16]:
#eliminar los duplicados de los productos de chedraui 
import pandas as pd
df = pd.read_json('productosChedraui.json')
df.drop_duplicates(subset ="nombre", keep = 'first', inplace = True)
df.to_json('productosChedraui.json', orient='records')


In [17]:
#eliminar los duplicados de los productos de soriana
import pandas as pd
df = pd.read_json('productosSoriana.json')
df.drop_duplicates(subset ="nombre", keep = 'first', inplace = True)
df.to_json('productosSoriana.json', orient='records')


Ahora podemos ya almacenar en una base de datos las entradas --->

In [19]:
# almacenar el json en una base de datos
import pymongo
from pymongo import MongoClient
import datetime
import json

# conectar a la base de datos
client = MongoClient('localhost', 27017)
db = client['productos']
collection = db['productos']


# insertar el json productosChedraui.json en la base de datos
with open('productosChedraui.json') as f:
    file_data = json.load(f)
collection.insert_many(file_data)

# insertar el json productosSoriana.json en la base de datos
with open('productosSoriana.json') as f:
    file_data = json.load(f)
collection.insert_many(file_data)


#collection.insert_many(pChedraui)
#collection.insert_many(pSoriana)
